# Text-Guided Editing of Images (Using CLIP and StyleGAN)

In [1]:
!git clone https://github.com/orpatashnik/StyleCLIP.git

import os
os.chdir(f'./StyleCLIP')

!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

Cloning into 'StyleCLIP'...
remote: Enumerating objects: 778, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 778 (delta 119), reused 105 (delta 105), pack-reused 618 (from 1)
Receiving objects: 100% (778/778), 193.81 MiB | 13.76 MiB/s, done.
Resolving deltas: 100% (264/264), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.8 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-knvqwm4e
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-knvqwm4e
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.9 MB/s eta 0:00:00

In [2]:
from utils import ensure_checkpoint_exists
from mapper.scripts.inference import run

We provide several pretrained mappers. We also provide sample latent codes, of 6 celebs. 'n_images' allows to choose the number of latent codes (from the path provided in 'latent_path') that will be edited. If n_images is None then all the latent codes will be edited.

In [3]:
meta_data = {
  'afro': ['afro', False, False, True],
  'angry': ['angry', False, False, True],
  'Beyonce': ['beyonce', False, False, False],
  'bobcut': ['bobcut', False, False, True],
  'bowlcut': ['bowlcut', False, False, True],
  'curly hair': ['curly_hair', False, False, True],
  'Hilary Clinton': ['hilary_clinton', False, False, False],
  'Jhonny Depp': ['depp', False, False, False],
  'mohawk': ['mohawk', False, False, True],
  'purple hair': ['purple_hair', False, False, False],
  'surprised': ['surprised', False, False, True],
  'Taylor Swift': ['taylor_swift', False, False, False],
  'trump': ['trump', False, False, False],
  'Mark Zuckerberg': ['zuckerberg', False, False, False]
}

edit_type = 'surprised' #@param ['afro', 'angry', 'Beyonce', 'bobcut', 'bowlcut', 'curly hair', 'Hilary Clinton', 'Jhonny Depp', 'mohawk', 'purple hair', 'surprised', 'Taylor Swift', 'trump', 'Mark Zuckerberg']
edit_id = meta_data[edit_type][0]
os.makedirs("mapper/pretrained", exist_ok=True)
ensure_checkpoint_exists(f"mapper/pretrained/{edit_id}.pt")
latent_path = "example_celebs.pt" #@param {type:"string"}
if latent_path == "example_celebs.pt":
  ensure_checkpoint_exists("example_celebs.pt")
n_images =  1#@param

args = {
    "work_in_stylespace": False,
    "exp_dir": "results/",
    "checkpoint_path": f"mapper/pretrained/{edit_id}.pt",
    "couple_outputs": True,
    "mapper_type": "LevelsMapper",
    "no_coarse_mapper": meta_data[edit_type][1],
    "no_medium_mapper": meta_data[edit_type][2],
    "no_fine_mapper": meta_data[edit_type][3],
    "stylegan_size": 1024,
    "test_batch_size": 1,
    "latents_test_path": latent_path,
    "test_workers": 1,
    "n_images": n_images
}


Downloading...
From (original): https://drive.google.com/uc?id=1F-mPrhO-UeWrV1QYMZck63R43aLtPChI
From (redirected): https://drive.google.com/uc?id=1F-mPrhO-UeWrV1QYMZck63R43aLtPChI&confirm=t&uuid=ca44a91c-4a79-4626-a328-57942b9f1e3d
To: /content/StyleCLIP/mapper/pretrained/surprised.pt
100%|██████████| 141M/141M [00:03<00:00, 35.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1VL3lP4avRhz75LxSza6jgDe-pHd2veQG
To: /content/StyleCLIP/example_celebs.pt
100%|██████████| 222k/222k [00:00<00:00, 94.6MB/s]


In [4]:
from argparse import Namespace
run(Namespace(**args))

Loading from checkpoint: mapper/pretrained/surprised.pt


  0%|          | 0/6 [00:00<?, ?it/s]


TypeError: make_grid() got an unexpected keyword argument 'range'

In [ ]:
from PIL import Image
result = Image.open(f"results/inference_results/00000.jpg")
result = result.resize((int(result.width * 0.5), int(result.height * 0.5)))
grid = Image.new("RGB", (result.width, result.height * n_images))
grid.paste(result, (0, 0))
for i in range(1, n_images):
  result = Image.open(f"results/inference_results/{str(i).zfill(5)}.jpg")
  result = result.resize((int(result.width * 0.5), int(result.height * 0.5)))
  grid.paste(result, (0, int(result.height * i)))
grid